In [ ]:
import cv2
import re
import imutils
import pandas as pd
import numpy as np
import math

In [ ]:
MODE = "picture" # "picture", "video" or "stream"

In [ ]:
WEBCAM_MODEL = "Microsoft LifeCam Cinema"
WEBCAM_HORIZONTAL_RESOLUTION = 1280
WEBCAM_VERTICAL_RESOLUTION = 720
WEBCAM_FPS = 30.0

In [ ]:
MILLISECONDS_BETWEEN_FRAMES = 750.0 if MODE == "picture" else 1000 / WEBCAM_FPS

In [ ]:
cap = cv2.ImageCapture(0) if MODE == "picture" else cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WEBCAM_HORIZONTAL_RESOLUTION)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, WEBCAM_VERTICAL_RESOLUTION)

In [ ]:
# from tangram_app.metrics import get_classification_report_pics
# from tangram_app.predictions import get_predictions_with_distances, get_predictions
# from tangram_app.processing import preprocess_img_2 as preprocess_img
# from tangram_app.tangram_game import tangram_game

In [ ]:
def crop(img, side="left"):
    """
    crop the left or right side of the image

    Parameters:
    - img : OpenCV
    - side : left / right

    Returns : OpenCV image
    """

    if side is None:
        return img

    assert side in ["left", "right"], "not a valid side"

    # we take only 55% of the frame either left or right side
    width_img = img.shape[1]
    box_width = int(width_img * 0.55)

    if side == "left":
        img = img[:, :box_width]
    else:
        box_width = width_img - box_width
        img = img[:, box_width:width_img]

    return img

In [ ]:
def blur(img, strength_blur=7, sensitivity_to_light=50):
    """
    This function takes a cv image as input, turns it into grayscale and blurs it. Used before finding contours to erase the white space between individual shapes.
    Parameters :

    origin_img = OpenCV image
    strength_blur = blur parameter (the greater the more "blurred")
    """

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # binarize img
    if sensitivity_to_light != "ignore":
        gray[gray > sensitivity_to_light] = 0
    blurred = cv2.medianBlur(gray, strength_blur)
    image_blurred = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY)[1]
    return image_blurred

In [ ]:
def get_contours(image):
    """
    This function retrieves an image's openCV contours.

    Parameters :
    image = OpenCV image
    """

    cnts = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    return cnts

In [ ]:
def display_contour(cnts, img):
    """
    display the contour of the image


    Parameters :
    cnts : contours of the forms in the image
    img : OpenCV image
    """
    for c in cnts:
        cv2.drawContours(img, [c], -1, (0, 255, 0), 2)

    img = imutils.resize(img, width=1200)
    cv2.imshow("Image", img)
    cv2.moveWindow("Image", 30, 30)
    cv2.waitKey(0)

In [ ]:
def find_moments(cnts, filename=None, hu_moment=True):
    """
    this function returns the shape's Moments or Hu Moments.

    Parameters :
    cnts : contours of an image
    filename : name of the image, useful when target name needed
    hu_moment : returns Hu Moments. if set to False, returns only the Moments

    Returns : Moments or Hu moments
    """

    lst_moments = [cv2.moments(c) for c in cnts]  # retrieve moments of all shapes identified
    lst_areas = [i["m00"] for i in lst_moments]  # retrieve areas of all shapes

    try:
        max_idx = lst_areas.index(max(lst_areas))  # select shape with the largest area

        if hu_moment:  # if we want the Hu moments
            HuMo = cv2.HuMoments(lst_moments[max_idx])  # grab humoments for largest shape
            if filename:
                HuMo = np.append(HuMo, filename)
            return HuMo

        # if we want to get the moments
        Moms = lst_moments[max_idx]
        if filename:
            Moms["target"] = filename
        return Moms
    except Exception as e:
        return []  # predictions impossible

In [ ]:
def dist_humoment(hu1, hu2):
    """
    return sum of euclidienne distance,sum of squart of pow difference
    """
    distance = np.linalg.norm(hu1 - hu2)
    return distance

In [ ]:
def detect_shapes(cnts, image):
    """
    This function detects all triangle and rectangle shapes in the image

    Parameters :
    @cnts: contours that previous function returns
    @image: image that we have preprocessed
    """
    cnts_output = []

    for cnt in cnts:
        perimetre = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * perimetre, True)

        area = cv2.contourArea(cnt)
        img_area = image.shape[0] * image.shape[1]
        # print("image area", img_area)
        if area / img_area > 0.005:
            # for triangle, if the shape has 3 angles
            if len(approx) == 3:
                cnts_output.append(cnt)

            # for quadrilater, if the shape has 4 angles
            elif len(approx) == 4:
                (x, y, w, h) = cv2.boundingRect(approx)

                ratio = w / float(h)
                # if the ratio is correct, we take this shape as a quadrilateral
                if ratio >= 0.33 and ratio <= 3:
                    cnts_output.append(cnt)

    return cnts_output

In [ ]:
def delete_isolated_shapes(formes, threshold=10):
    """
    Delete all shapes if the most distance between the shape to all shapes is bigger than the threshold
    
    Parameters :
    @formes: the dictionnay containing keys of shapes and the array containing the contour
    @threshold: the threshold of distance between shapes
    """
    mindistances = {}
    # Save all min distances by shape to a dictionary
    for keys1, values1 in formes.items():
        mindistances[keys1] = []
        for i in range(len(values1)):
            M1 = cv2.moments(values1[i])
            center_i_x, center_i_y = int(M1["m10"] / M1["m00"]), int(M1["m01"] / M1["m00"])
            min_distance = 99999999
            for keys2, values2 in formes.items():
                for j in range(len(values2)):
                    # if keys1 != keys2 and i != j:
                    M2 = cv2.moments(values2[j])
                    center_j_x, center_j_y = int(M2["m10"] / M2["m00"]), int(M2["m01"] / M2["m00"])
                    distance = math.sqrt(pow(center_i_x - center_j_x, 2) + pow(center_i_y - center_j_y, 2))
                    if distance < min_distance and distance > 0:
                        min_distance = distance
            mindistances[keys1].append(min_distance)

    # we take all shapes if this shape is smaller than threshold
    forme_output = {}
    for keys, values in mindistances.items():
        forme_output[keys] = []
        for i in range(len(values)):
            if mindistances[keys][i] < threshold:
                forme_output[keys].append(formes[keys][i])

    return forme_output

In [ ]:
def distance_shapes(contours):
    """
    In the first step this function separates all shapes in 5 different shapes: small triangle, midlle triangle, big triangle, square and parallelogram
    In the second step it calculates the perimeters and centers of all shapes, and remove duplicate ones

    Parameter:
    @contours: the cv2.contours that returns last function

    Returns : centers and perimeters of all shapes
    """

    formes = {"triangle": [], "squart": [], "parallelo": []}

    for cnt in contours:
        perimetre = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.05 * perimetre, True)
        # if the shape has 3 angles we consider this shape is a triangle
        if len(approx) == 3:
            formes["triangle"].append(cnt)
        # if the shape has 4 angles we consider this shape is a quadrilateral
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ratio = w / float(h)

            # if the quadrilateral has this ratio between height and width, we consider this is a square
            # if ratio >= 0.9 and ratio <= 1.1:
            if ratio >= 0.7 and ratio <= 1.3:
                formes["squart"].append(cnt)

            # if the quadrilateral has this ratio between height and width, we consider this is a parallelogram
            # elif (ratio >= 0.3 and ratio <= 3.3):
            elif ratio >= 0.2 and ratio <= 5:
                formes["parallelo"].append(cnt)

    # Remove isolated shapes
    formes = delete_isolated_shapes(formes, 200)

    # dictionnay to take barycenters of all shapes
    centers = {"smallTriangle": [], "middleTriangle": [], "bigTriangle": [], "squart": [], "parallelo": []}

    # dictionnay to take perimeters of all shapes
    perimeters = {"smallTriangle": [], "middleTriangle": [], "bigTriangle": [], "squart": [], "parallelo": []}

    if len(formes["triangle"]) > 0:
        # we detecte the size of trianlge, we compare the area of triangle to the unique square's, if we detect we have just one parallelogram

        if len(formes["squart"]) == 1:
            areaSquart = cv2.contourArea(formes["squart"][0])
            for triangle in formes["triangle"]:
                triangle_perimeter = cv2.arcLength(triangle, True)
                M = cv2.moments(triangle)
                triangle_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                areaTriangle = cv2.contourArea(triangle)
                rapport = areaTriangle / areaSquart
                if rapport < 0.5:
                    centers["smallTriangle"].append(triangle_center)
                    perimeters["smallTriangle"].append(triangle_perimeter)
                elif rapport < 1.15:
                    centers["middleTriangle"].append(triangle_center)
                    perimeters["middleTriangle"].append(triangle_perimeter)
                else:
                    centers["bigTriangle"].append(triangle_center)
                    perimeters["bigTriangle"].append(triangle_perimeter)

        # Comparer la taille des triangle à parallélograme unique
        elif len(formes["parallelo"]) == 1:
            areaSquart = cv2.contourArea(formes["parallelo"][0])

            for triangle in formes["triangle"]:

                triangle_perimeter = cv2.arcLength(triangle, True)
                M = cv2.moments(triangle)
                triangle_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                areaTriangle = cv2.contourArea(triangle)
                rapport = areaTriangle / areaSquart
                if rapport < 0.6:
                    centers["smallTriangle"].append(triangle_center)
                    perimeters["smallTriangle"].append(triangle_perimeter)
                elif rapport < 1.5:
                    centers["middleTriangle"].append(triangle_center)
                    perimeters["middleTriangle"].append(triangle_perimeter)
                else:
                    centers["bigTriangle"].append(triangle_center)
                    perimeters["bigTriangle"].append(triangle_perimeter)

        else:
            # else we compare between the bigger triangle and the smaller triangle
            triangleArea = [cv2.contourArea(triangle) for triangle in formes["triangle"]]
            min_triangle_area = min(triangleArea)
            max_triangle_area = max(triangleArea)

            # Si c'est plus grand triangle avec plus petit triangle
            if max_triangle_area / min_triangle_area > 5:

                for triangle in formes["triangle"]:
                    triangle_perimeter = cv2.arcLength(triangle, True)
                    M = cv2.moments(triangle)
                    triangle_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                    if max_triangle_area / cv2.contourArea(triangle) > 5:
                        centers["smallTriangle"].append(triangle_center)
                        perimeters["smallTriangle"].append(triangle_perimeter)

                    elif max_triangle_area / cv2.contourArea(triangle) > 2:
                        centers["middleTriangle"].append(triangle_center)
                        perimeters["middleTriangle"].append(triangle_perimeter)

                    else:
                        centers["bigTriangle"].append(triangle_center)
                        perimeters["bigTriangle"].append(triangle_perimeter)

        # for case of square
        for squart in formes["squart"]:
            squart_perimeter = cv2.arcLength(squart, True)
            M = cv2.moments(squart)
            squart_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            centers["squart"].append(squart_center)
            perimeters["squart"].append(squart_perimeter)

        # for case of parallelogram
        for parallelo in formes["parallelo"]:
            parallelo_perimeter = cv2.arcLength(parallelo, True)
            M = cv2.moments(parallelo)
            parallelo_center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            centers["parallelo"].append(parallelo_center)
            perimeters["parallelo"].append(parallelo_perimeter)

    # Remove duplicate shapes
    centers2 = {}
    perimeters2 = {}

    for key, values in centers.items():
        centers2[key] = []
        perimeters2[key] = []
        if (len(values)) > 0:
            centers2[key].append(centers[key][0])
            perimeters2[key].append(perimeters[key][0])
            for i in range(1, len(values)):
                x1, y1 = values[i]
                isDistanceBigger = True
                for j in range(i):
                    x2, y2 = values[j]
                    distance = round(math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2)), 2)
                    if distance < 20:
                        isDistanceBigger = False
                if isDistanceBigger:
                    centers2[key].append(centers[key][i])
                    perimeters2[key].append(perimeters[key][i])

    return centers2, perimeters2

In [ ]:
def ratio_distance(centers, perimeters):
    """
    This function calculate all ratios of distances between shapes with a shape's side
    Author: @Gautier
    ==================================================
    Parameters
    @centers: an array of centers,  a center point is a tuple of 2 numbers: absciss, ordinate, and a
    @perimeters: a dictionnay of perimeters, it has keys of all shape's name

    Returns : distances of all shapes between each other
    """
    distances = {}

    for forme1, centers1 in centers.items():
        inx = []
        for i in range(len(centers1)):
            for forme2, centers2 in centers.items():
                for j in range(len(centers2)):
                    if forme1 + str(i) != forme2 + str(j):
                        if (forme1 + "_" + str(i + 1) + "-" + forme2 + "_" + str(j + 1) not in list(distances)) and (
                            forme2 + "_" + str(j + 1) + "-" + forme1 + "_" + str(i + 1) not in list(distances)
                        ):
                            inx.append(str(i) + str(j))
                            x1, y1 = centers1[i]
                            x2, y2 = centers2[j]
                            absolute_distance = round(math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2)), 2)

                            if len(perimeters["squart"]) > 0:
                                squart_perimeter = perimeters["squart"][0]
                                relative_distance = round(
                                    absolute_distance / squart_perimeter * (4 * math.sqrt(1 / 8)), 2
                                )
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = relative_distance

                            elif len(perimeters["parallelo"]) > 0:
                                parallelo_perimeter = perimeters["parallelo"][0]
                                relative_distance = round(
                                    absolute_distance / parallelo_perimeter * (2 * math.sqrt(1 / 8) + 1), 2
                                )
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = relative_distance

                            elif len(perimeters["smallTriangle"]) > 0:
                                smallTriangle_perimeter = perimeters["smallTriangle"][0]
                                relative_distance = round(
                                    absolute_distance / smallTriangle_perimeter * (2 * math.sqrt(1 / 8) + 1 / 2), 2
                                )
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = relative_distance

                            elif len(perimeters["middleTriangle"]) > 0:
                                middleTriangle_perimeter = perimeters["middleTriangle"][0]
                                relative_distance = round(
                                    absolute_distance / middleTriangle_perimeter * (1 + math.sqrt(1 / 2)), 2
                                )
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = relative_distance

                            elif len(perimeters["bigTriangle"]) > 0:
                                bigTriangle_perimeter = perimeters["bigTriangle"][0]
                                relative_distance = round(
                                    absolute_distance / bigTriangle_perimeter * (1 + 2 * math.sqrt(1 / 2)), 2
                                )
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = relative_distance

                            else:
                                distances[forme1 + "-" + forme2 + "_" + str(i + 1) + str(j + 1)] = 0
    return distances

In [ ]:
def sorted_distances(distances):
    """
    This function sorts the distances between the same relationship of two shapes.

    :param distances: Dictionary of distances between all pairs of shapes, e.g., {"smallTriangle-middleTriangle_11": [0.5], "smallTriangle-middleTriangle_21": [0.4]}.
    :type distances: dict

    :return: A dictionary of distances between all pairs of shapes, ordered by distance and with keys renamed by the number of the same relation, e.g., {"smallTriangle-middleTriangle1": [0.4], "smallTriangle-middleTriangle2": [0.5]}.
    :rtype: dict
    """
    data_distances = {
        "smallTriangle-smallTriangle": [],
        "smallTriangle-middleTriangle": [],
        "smallTriangle-bigTriangle": [],
        "smallTriangle-squart": [],
        "smallTriangle-parallelo": [],
        "middleTriangle-bigTriangle": [],
        "middleTriangle-squart": [],
        "middleTriangle-parallelo": [],
        "bigTriangle-bigTriangle": [],
        "bigTriangle-squart": [],
        "bigTriangle-parallelo": [],
        "squart-parallelo": [],
    }

    keys = ["smallTriangle", "middleTriangle", "bigTriangle", "squart", "parallelo"]

    liste = []
    for i in range(len(keys)):
        for j in range(i):
            liste.append(keys[j] + "-" + keys[i])
    liste.append("smallTriangle-smallTriangle")
    liste.append("bigTriangle-bigTriangle")

    for key, value in distances.items():

        for title in liste:
            if title in key:
                data_distances[title].append(value)
        for title in liste:
            data_distances[title] = sorted(data_distances[title])

    if len(data_distances["smallTriangle-smallTriangle"]) > 1:
        del data_distances["smallTriangle-smallTriangle"][1]

    if len(data_distances["bigTriangle-bigTriangle"]) > 1:
        del data_distances["bigTriangle-bigTriangle"][1]

    data_sortered = {}

    for key, value in data_distances.items():
        if len(value) > 1:
            for i in range(len(value)):
                data_sortered[key + str(i + 1)] = value[i]
        elif len(value) == 1:
            data_sortered[key + str(1)] = value[0]
    return data_sortered

In [ ]:
def mse_distances(data, sorted_dists):
    """
    This function returns a list of rmse that we have between our tangram with all classes
    
    Parameters:
    @data: the dataframe containing all distances of shapes of all classes
    @sorted_dists: the dictionnay containing our tangram's shape distances
    """
    mses = []
    for i in range(data.shape[0]):
        ligne = data.iloc[i]
        mses.append(
            round(
                math.sqrt(
                    sum([
                        pow(ligne[index] - sorted_dists[index], 2)
                        for index, _ in sorted_dists.items()
                        if index in ligne.keys()
                    ])
                ),
                3,
            )
        )
    return mses

In [ ]:
def extract_triangles_squares(cnts, img):
    """
    This function extracts only the triangles, squares and parallelograms from a list of contours and hence removes 'noise' items.
    To do so, it calls OpenCV's appoxPolyDp function to identify which contours have either 3 or 4 summits then returns those that
    fit area and/or width-height ratio criteria.

    Parameters :
    cnts_output = list of contours from cv2.findContours
    origin_img = OpenCV image
    """

    cnts_output = []
    out_image = np.zeros(img.shape, img.dtype)

    for idx, cnt in enumerate(cnts):
        perimetre = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.02 * perimetre, True)
        area = cv2.contourArea(cnt)
        img_area = img.shape[0] * img.shape[1]

        if area / img_area > 0.0005:
            # for triangle
            if len(approx) == 3:
                cnts_output.append(cnt)
                cv2.drawContours(img, [cnt], -1, (50, 255, 50), 3)
                cv2.fillPoly(img, pts=[cnt], color=(50, 255, 50))  # ??
            # for quadrilater
            elif len(approx) == 4:
                (x, y, w, h) = cv2.boundingRect(approx)
                ratio = w / float(h)
                if ratio >= 0.2 and ratio <= 4:
                    cnts_output.append(cnt)
                    cv2.drawContours(img, [cnt], -1, (50, 255, 50), 3)
                    cv2.fillPoly(img, pts=[cnt], color=(50, 255, 50))

    return cnts_output, out_image

In [ ]:
def preprocess_img(origin_img, side):
    """
    this function takes a cv image as input, crops it to focus on the right/left side, detects edges using OpenCV's Canny,
    dilates the resulting contours to make them continuous, binarizes the image and finds contours.
    It then calls the extract_triangles_squares_function to keep only regular geometric shapes (and eliminate all "noise" contours)

    Parameters :
    origin_img = OpenCV image
    side = process either left/right side or full frame. Passed from the CLI argument
    """

    origin_img = crop(origin_img, side=side)
    img = cv2.Canny(origin_img, 30, 300)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    img = cv2.dilate(img, kernel)
    img = cv2.threshold(img.copy(), 0, 255, cv2.THRESH_BINARY)[1]
    cnts, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnts_output, triangle_squares_img = extract_triangles_squares(cnts, img)
    return cnts_output, origin_img

In [ ]:
def save_moments(images, directory):
    """
    compute moments / hu moments for all images in our dataset

    Parameters :
    images : dict with images names and paths

    Return : save moments and hu_moments into CSV files and return them as Pandas dataframe
    """

    hu_moments = []
    moments = []

    for image_name, image_path in images:
        img_cv = cv2.imread(image_path)

        pattern = re.compile(r"([a-zA-Z]+)_\d{1,2}_(\w+)")

        result = pattern.search(image_path)

        if result:
            side = result.group(2)
        else:
            side = None

        cnts, img = preprocess_img(img_cv, side=side)

        hu_moments.append(find_moments(cnts, image_name))
        moments.append(find_moments(cnts, image_name, hu_moment=False))

        hu_moments_df = pd.DataFrame(hu_moments)
        hu_moments_df.to_csv(directory + "/hu_moments.csv", index=False)

        moments_df = pd.DataFrame(moments)
        moments_df.to_csv(directory + "/moments.csv", index=False)

    return hu_moments_df, moments_df

In [ ]:
def create_all_types_distances(link):
    """
    Create a csv file of all distances of shapes of all ours classes

    Parameters :
    @link: directory to save the csv file
    """
    images = [
        "bateau.jpg",
        "bol.jpg",
        "chat.jpg",
        "coeur.jpg",
        "cygne.jpg",
        "lapin.jpg",
        "maison.jpg",
        "marteau.jpg",
        "montagne.jpg",
        "pont.jpg",
        "renard.jpg",
        "tortue.jpg",
    ]

    data = pd.DataFrame(
        columns=[
            "smallTriangle-smallTriangle1",
            "smallTriangle-middleTriangle1",
            "smallTriangle-middleTriangle2",
            "smallTriangle-bigTriangle1",
            "smallTriangle-bigTriangle2",
            "smallTriangle-bigTriangle3",
            "smallTriangle-bigTriangle4",
            "smallTriangle-squart1",
            "smallTriangle-squart2",
            "smallTriangle-parallelo1",
            "smallTriangle-parallelo2",
            "middleTriangle-bigTriangle1",
            "middleTriangle-bigTriangle2",
            "middleTriangle-squart1",
            "middleTriangle-parallelo1",
            "bigTriangle-bigTriangle1",
            "bigTriangle-squart1",
            "bigTriangle-squart2",
            "bigTriangle-parallelo1",
            "bigTriangle-parallelo2",
            "squart-parallelo1",
            "classe",
        ]
    )

    data = pd.Series()
    for im in images:
        img_cv = cv2.imread("data/tangrams/" + im)
        cnts, img = preprocess_img(img_cv, side=None)
        cnts_forms = detect_shapes(cnts, img)
        centers, perimeters = distance_shapes(cnts_forms)
        distances = ratio_distance(centers, perimeters)
        sorted_dists = sorted_distances(distances)
        classe = im.split(".")[0]
        sorted_dists["classe"] = classe
        data = data.append(sorted_dists, ignore_index=True)

    data.to_csv(link, sep=";")

In [ ]:
def get_predictions_with_distances(img_cv, side, prepro):
    """
    This function take in input a image and return a dictionnay of shape distances

    Parameters:
    @img_cv: input image
    @side: it take the position of the table, if side is left we take just the left side of table, right we take the right side
    @prepro: function of preprocessing
    """
    cnts, cropped_img = prepro(img_cv, side=side)

    for c in cnts:
        cv2.drawContours(cropped_img, [c], -1, (50, 255, 50), 2)

    centers, perimeters = distance_shapes(cnts)
    distances = ratio_distance(centers, perimeters)
    sorted_dists = sorted_distances(distances)

    # get distances
    data = pd.read_csv("data/tangram_properties/data.csv", sep=";")
    mses = np.array(mse_distances(data, sorted_dists))

    # get proba
    if np.all((mses == 0)):
        return None

    # fix the issue where some proba == 0
    mses[mses == 0] = 0.0001
    proba = np.round(1 / mses / np.sum(1 / mses), 3)

    # get probabilities
    probas_labelled = data[["classe"]].rename(columns={"classe": "target"})
    probas_labelled.loc[:, "proba"] = proba

    probas_labelled = probas_labelled.sort_values(by=["proba"], ascending=False).reset_index(drop=True)

    # returns sorted probas
    return probas_labelled

In [ ]:
def get_predictions(image, prepro, side, hu_moments_dataset="data/tangram_properties/hu_moments.csv"):
    """
    compare moments of a frame with the hu moments of our dataset images

    Parameters :
    image : OpenCV image
    hu_moments : dataset with the humoments of each class
    target : name of the classes
    side : which side should be analyzed - left / right / full image

    Return : print the probabilities to belong to each class in descending order (Pandas DataFrame)
    """

    # get dataset
    hu_moments = pd.read_csv(hu_moments_dataset)
    target = hu_moments.iloc[:, -1]

    # Our operations on the frame come here
    cnts, img = prepro(image, side=side)

    HuMo = find_moments(cnts)

    if len(HuMo) == 0:
        return None  # the image can't be processed, so empty predictions

    # with the hu_moments we can get the predictions
    HuMo = np.hstack(HuMo)

    # get distances
    dist = hu_moments.apply(lambda row: dist_humoment(HuMo, row.values[:-1]), axis=1)
    dist_labelled = pd.concat([dist, target], axis=1)
    dist_labelled.columns = ["distance", "target"]

    # get probabilities
    dist_labelled["proba"] = round((1 / dist_labelled["distance"]) / np.sum(1 / dist_labelled["distance"], axis=0), 2)
    probas = dist_labelled.sort_values(by=["proba"], ascending=False)[["target", "proba"]].reset_index(drop=True)

    # sorted probabilities
    return probas